In [2]:
# Perform imports here:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from pytz import timezone
from sqlalchemy import create_engine
import math
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN
import gc
from joblib import Parallel, delayed
import joblib

In [ ]:
#df = pd.read_pickle("OLI_SELL.pk")
df = pd.read_pickle("GSS_SELL.pk")
#df = pd.read_pickle("HMA_SELL.pk")

In [ ]:
df.SELL_AMOUNT.value_counts().reset_index().sort_values("index",ascending=False)

In [ ]:
df.SELL_AMOUNT.value_counts().reset_index().sort_values("index",ascending=False)[25:]

In [ ]:
df.SELL_AMOUNT.value_counts().reset_index().sort_values("index",ascending=False)[50:]

In [ ]:
df.SELL_AMOUNT.max()

In [ ]:
df.head()

In [ ]:
n = 500
out = df[df.SELL_AMOUNT>n].STOCK_ID.unique()

In [ ]:
df[df.SELL_AMOUNT>n].groupby("BRAND").size()

In [ ]:
df[df.SELL_AMOUNT>n].groupby("BRAND").ITEM_ID.nunique()

In [ ]:
df[df.SELL_AMOUNT>n].groupby("BRAND").STOCK_ID.nunique()

In [ ]:
df[(df.STOCK_ID.isin(out))&(df.BRAND=="AP")].sort_values(["ITEM_ID", "STOCK_ID", "COLLECT_DAY"])

In [ ]:
df[(df.STOCK_ID.isin(out))&(df.BRAND=="LG")].sort_values(["ITEM_ID", "STOCK_ID", "COLLECT_DAY"])

In [ ]:
df[(df.STOCK_ID.isin(out))&(df.BRAND=="LOREAL")].sort_values(["ITEM_ID", "STOCK_ID", "COLLECT_DAY"])

In [ ]:
df[(df.STOCK_ID.isin(out))&(df.BRAND=="LVMH")].sort_values(["ITEM_ID", "STOCK_ID", "COLLECT_DAY"])

## 루이비똥 봐보자 ...

In [ ]:
n

In [ ]:
lvmh = df[(df.SELL_AMOUNT<n)&(df.BRAND=="LVMH")]

In [ ]:
lvmh.SELL.sort_values(ascending=False)

In [ ]:
lvmh.loc[lvmh.SELL.sort_values(ascending=False).index[1:100]].SELL_AMOUNT

In [ ]:
lvmh[lvmh.STOCK_ID==lvmh.loc[504031].STOCK_ID]

In [ ]:
lvmh[(lvmh.SELL_AMOUNT < 480)|(lvmh.SELL_AMOUNT > 500)].SELL.sum()

In [ ]:
temp = lvmh.SELL_AMOUNT.value_counts().sort_values(ascending=False)
temp2 = temp[temp>1].reset_index()
shit = temp2[temp2["index"]>100]['index']

In [ ]:
lvmh[-lvmh.SELL_AMOUNT.isin(shit)].SELL.sum()

## 판매량 계산

In [ ]:
df.groupby("BRAND").SELL.sum().reset_index()

In [ ]:
df[df.SELL_AMOUNT<500].groupby("BRAND").SELL.sum().reset_index()

In [ ]:
df[df.SELL_AMOUNT<100].groupby("BRAND").SELL.sum().reset_index()

## 일단 정리하자~~

In [3]:
OLI = pd.read_pickle("OLI_SELL.pk")
GSS = pd.read_pickle("GSS_SELL.pk")
HMA = pd.read_pickle("HMA_SELL.pk")

In [43]:
OLI = OLI[OLI.SELL_AMOUNT<500]
HMA = HMA[HMA.SELL_AMOUNT<500]
GSS = GSS[GSS.SELL_AMOUNT<500]

In [44]:
n = 500
lvmh = GSS[(GSS.SELL_AMOUNT<n)&(GSS.BRAND=="LVMH")]
temp = lvmh.SELL_AMOUNT.value_counts().sort_values(ascending=False)
temp2 = temp[temp>1].reset_index()
shit = temp2[temp2["index"]>100]['index']

In [45]:
GSS[-((GSS.BRAND=="LVMH")&(GSS.SELL_AMOUNT.isin(shit)))].groupby("BRAND").SELL.sum()

BRAND
AP        4.869031e+09
LG        4.793041e+09
LOREAL    1.969938e+09
LVMH      7.868710e+08
init      4.364687e+10
Name: SELL, dtype: float64

In [46]:
HMA.groupby("BRAND").SELL.sum()

BRAND
AP        2.055956e+09
LG        1.048695e+09
LOREAL    2.701683e+09
LVMH      4.114830e+08
init      7.396535e+09
Name: SELL, dtype: float64

In [47]:
OLI.groupby("BRAND").SELL.sum()

BRAND
AP        1.935789e+08
LG        5.211606e+08
LOREAL    4.286401e+08
LVMH      9.826590e+07
init      1.052347e+10
Name: SELL, dtype: float64

In [48]:
GSS=GSS[-((GSS.BRAND=="LVMH")&(GSS.SELL_AMOUNT.isin(shit)))]

### 이상한 놈들 제거

In [49]:
GSS = GSS[-GSS.ITEM_ID.isin([1907459,8427631,447103,1909086,11412866, 8866616])]

In [50]:
OLI.to_pickle("OLI0713.pk")
HMA.to_pickle("HMA0713.pk")
GSS.to_pickle("GSS0713.pk")

## 정리한 거 계산 ..

In [51]:
OLI_pivot = OLI.groupby(["BRAND", "CATE_NAMES"]).SELL.sum().reset_index().pivot(index="CATE_NAMES", columns="BRAND", values="SELL")
HMA_pivot = HMA.groupby(["BRAND", "CATE_NAMES"]).SELL.sum().reset_index().pivot(index="CATE_NAMES", columns="BRAND", values="SELL")
GSS_pivot = GSS.groupby(["BRAND", "CATE_NAMES"]).SELL.sum().reset_index().pivot(index="CATE_NAMES", columns="BRAND", values="SELL")

In [52]:
OLI_pivot["SITE_NAME"] = "Oliveyoung"
GSS_pivot["SITE_NAME"] = "GSSHOP"
HMA_pivot["SITE_NAME"] = "HMALL"

In [53]:
report = pd.concat([OLI_pivot, GSS_pivot, HMA_pivot]).reset_index()
report = report.sort_values(["CATE_NAMES", "SITE_NAME"]).reset_index(drop=True)

In [54]:

report[["CATE_NAMES", "SITE_NAME", "AP", "LG", "LOREAL", "LVMH", "init"]].to_csv("REPORT0717.csv", index=False)
report[["AP", "LG", "LOREAL", "LVMH", "init", "SITE_NAME"]].groupby("SITE_NAME").sum().to_csv("REPORT20713.csv")

## 연습장

In [ ]:
old = pd.read_pickle("OLI_SELL_old.pk")

In [47]:
#GSS[(GSS.BRAND=='LVMH')&(GSS.NORMAL_PRICE_SITE==25500)]
tmp = GSS[GSS.GOODS_NAME=='[메이블린 뉴욕]_핏미 파운데이션+컨실러 2종세트']

In [48]:
tmp

,SELL_AMOUNT,STOCK_AMOUNT,REVISE_STOCK_AMOUNT,STOCK_ID,ITEM_ID,UPT_DT,COLLECT_DAY,UPT_ID,REG_DT,ITEM_NUM,GOODS_NAME,COLLECT_URL,NORMAL_PRICE_SITE,BRAND_NAME,SITE_NO,CATE_NAMES,BRAND,SELL
657775,0,496,496,30577519001,9933893,2018-07-13 13:42:50,2018-06-01,FILTER ALGO,2018-06-01,30577519,[메이블린 뉴욕]_핏미 파운데이션+컨실러 2종세트,http://www.gsshop.com/prd/prd.gs?prdid=30577519,23200,메이블린 뉴욕,1,메이크업>베이스,LOREAL,0
657776,-0,None,496,30577519001,9933893,2018-07-13 13:42:50,2018-06-02,FILTER ALGO,2018-06-02,30577519,[메이블린 뉴욕]_핏미 파운데이션+컨실러 2종세트,http://www.gsshop.com/prd/prd.gs?prdid=30577519,23200,메이블린 뉴욕,1,메이크업>베이스,LOREAL,-0
657777,-0,None,496,30577519001,9933893,2018-07-13 13:42:50,2018-06-03,FILTER ALGO,2018-06-03,30577519,[메이블린 뉴욕]_핏미 파운데이션+컨실러 2종세트,http://www.gsshop.com/prd/prd.gs?prdid=30577519,23200,메이블린 뉴욕,1,메이크업>베이스,LOREAL,-0
657778,-0,496,496,30577519001,9933893,2018-07-13 13:42:50,2018-06-04,FILTER ALGO,2018-06-04,30577519,[메이블린 뉴욕]_핏미 파운데이션+컨실러 2종세트,http://www.gsshop.com/prd/prd.gs?prdid=30577519,23200,메이블린 뉴욕,1,메이크업>베이스,LOREAL,-0
657779,-0,None,496,30577519001,9933893,2018-07-13 13:42:50,2018-06-05,FILTER ALGO,2018-06-05,30577519,[메이블린 뉴욕]_핏미 파운데이션+컨실러 2종세트,http://www.gsshop.com/prd/prd.gs?prdid=30577519,23200,메이블린 뉴욕,1,메이크업>베이스,LOREAL,-0
657780,-0,None,496,30577519001,9933893,2018-07-13 13:42:50,2018-06-06,FILTER ALGO,2018-06-06,30577519,[메이블린 뉴욕]_핏미 파운데이션+컨실러 2종세트,http://www.gsshop.com/prd/prd.gs?prdid=30577519,23200,메이블린 뉴욕,1,메이크업>베이스,LOREAL,-0
657781,-0,496,496,30577519001,9933893,2018-07-13 13:42:50,2018-06-07,FILTER ALGO,2018-06-07,30577519,[메이블린 뉴욕]_핏미 파운데이션+컨실러 2종세트,http://www.gsshop.com/prd/prd.gs?prdid=30577519,23200,메이블린 뉴욕,1,메이크업>베이스,LOREAL,-0
657782,-0,496,496,30577519001,9933893,2018-07-13 13:42:50,2018-06-08,FILTER ALGO,2018-06-08,30577519,[메이블린 뉴욕]_핏미 파운데이션+컨실러 2종세트,http://www.gsshop.com/prd/prd.gs?prdid=30577519,23200,메이블린 뉴욕,1,메이크업>베이스,LOREAL,-0
657783,-0,496,496,30577519001,9933893,2018-07-13 13:42:50,2018-06-09,FILTER ALGO,2018-06-09,30577519,[메이블린 뉴욕]_핏미 파운데이션+컨실러 2종세트,http://www.gsshop.com/prd/prd.gs?prdid=30577519,23200,메이블린 뉴욕,1,메이크업>베이스,LOREAL,-0
657784,-0,496,496,30577519001,9933893,2018-07-13 13:42:50,2018-06-10,FILTER ALGO,2018-06-10,30577519,[메이블린 뉴욕]_핏미 파운데이션+컨실러 2종세트,http://www.gsshop.com/prd/prd.gs?prdid=30577519,23200,메이블린 뉴욕,1,메이크업>베이스,LOREAL,-0


In [55]:
aa = tmp.groupby("STOCK_ID").SELL_AMOUNT.sum()
aa

STOCK_ID
30577519001      0.0
30577519002    490.0
30577519003      0.0
30577519004      0.0
30577519005      0.0
30577519006      0.0
30577519007    481.0
30577519008      0.0
30577519009      0.0
30577519010      0.0
30577519011      1.0
30577519012    499.0
30577519013      0.0
30577519014      0.0
30577519015      0.0
30577519016      0.0
30577519017    498.0
30577519018      0.0
30577519019      1.0
30577519020      0.0
30577519021      0.0
30577519022    500.0
30577519023      0.0
30577519024      0.0
30577519025      0.0
30577519026      0.0
30577519027    500.0
30577519028      0.0
30577519029      0.0
30577519030      0.0
Name: SELL_AMOUNT, dtype: float64

In [54]:
aa[(aa>0)].index

Index(['30577519002', '30577519007', '30577519011', '30577519012',
       '30577519017', '30577519019', '30577519022', '30577519027'],
      dtype='object', name='STOCK_ID')

In [57]:
tmp.STOCK_ID.nunique()

30

In [62]:
tmp[tmp.STOCK_ID=='30577519010']

,SELL_AMOUNT,STOCK_AMOUNT,REVISE_STOCK_AMOUNT,STOCK_ID,ITEM_ID,UPT_DT,COLLECT_DAY,UPT_ID,REG_DT,ITEM_NUM,GOODS_NAME,COLLECT_URL,NORMAL_PRICE_SITE,BRAND_NAME,SITE_NO,CATE_NAMES,BRAND,SELL
658036,0,50,50,30577519010,9933893,2018-07-13 13:42:50,2018-06-01,FILTER ALGO,2018-06-01,30577519,[메이블린 뉴욕]_핏미 파운데이션+컨실러 2종세트,http://www.gsshop.com/prd/prd.gs?prdid=30577519,23200,메이블린 뉴욕,1,메이크업>베이스,LOREAL,0
658037,-0,None,50,30577519010,9933893,2018-07-13 13:42:50,2018-06-02,FILTER ALGO,2018-06-02,30577519,[메이블린 뉴욕]_핏미 파운데이션+컨실러 2종세트,http://www.gsshop.com/prd/prd.gs?prdid=30577519,23200,메이블린 뉴욕,1,메이크업>베이스,LOREAL,-0
658038,-0,None,50,30577519010,9933893,2018-07-13 13:42:50,2018-06-03,FILTER ALGO,2018-06-03,30577519,[메이블린 뉴욕]_핏미 파운데이션+컨실러 2종세트,http://www.gsshop.com/prd/prd.gs?prdid=30577519,23200,메이블린 뉴욕,1,메이크업>베이스,LOREAL,-0
658039,-0,50,50,30577519010,9933893,2018-07-13 13:42:50,2018-06-04,FILTER ALGO,2018-06-04,30577519,[메이블린 뉴욕]_핏미 파운데이션+컨실러 2종세트,http://www.gsshop.com/prd/prd.gs?prdid=30577519,23200,메이블린 뉴욕,1,메이크업>베이스,LOREAL,-0
658040,-0,None,50,30577519010,9933893,2018-07-13 13:42:50,2018-06-05,FILTER ALGO,2018-06-05,30577519,[메이블린 뉴욕]_핏미 파운데이션+컨실러 2종세트,http://www.gsshop.com/prd/prd.gs?prdid=30577519,23200,메이블린 뉴욕,1,메이크업>베이스,LOREAL,-0
658041,-0,None,50,30577519010,9933893,2018-07-13 13:42:50,2018-06-06,FILTER ALGO,2018-06-06,30577519,[메이블린 뉴욕]_핏미 파운데이션+컨실러 2종세트,http://www.gsshop.com/prd/prd.gs?prdid=30577519,23200,메이블린 뉴욕,1,메이크업>베이스,LOREAL,-0
658042,-0,50,50,30577519010,9933893,2018-07-13 13:42:50,2018-06-07,FILTER ALGO,2018-06-07,30577519,[메이블린 뉴욕]_핏미 파운데이션+컨실러 2종세트,http://www.gsshop.com/prd/prd.gs?prdid=30577519,23200,메이블린 뉴욕,1,메이크업>베이스,LOREAL,-0
658043,-0,50,50,30577519010,9933893,2018-07-13 13:42:50,2018-06-08,FILTER ALGO,2018-06-08,30577519,[메이블린 뉴욕]_핏미 파운데이션+컨실러 2종세트,http://www.gsshop.com/prd/prd.gs?prdid=30577519,23200,메이블린 뉴욕,1,메이크업>베이스,LOREAL,-0
658044,-0,50,50,30577519010,9933893,2018-07-13 13:42:50,2018-06-09,FILTER ALGO,2018-06-09,30577519,[메이블린 뉴욕]_핏미 파운데이션+컨실러 2종세트,http://www.gsshop.com/prd/prd.gs?prdid=30577519,23200,메이블린 뉴욕,1,메이크업>베이스,LOREAL,-0
658045,-0,50,50,30577519010,9933893,2018-07-13 13:42:50,2018-06-10,FILTER ALGO,2018-06-10,30577519,[메이블린 뉴욕]_핏미 파운데이션+컨실러 2종세트,http://www.gsshop.com/prd/prd.gs?prdid=30577519,23200,메이블린 뉴욕,1,메이크업>베이스,LOREAL,-0


In [59]:
tmp[(tmp.STOCK_ID.isin(aa[(aa>0)].index))&(tmp.SELL_AMOUNT>70)]
    #&(tmp.REVISE_STOCK_AMOUNT<98)]

,SELL_AMOUNT,STOCK_AMOUNT,REVISE_STOCK_AMOUNT,STOCK_ID,ITEM_ID,UPT_DT,COLLECT_DAY,UPT_ID,REG_DT,ITEM_NUM,GOODS_NAME,COLLECT_URL,NORMAL_PRICE_SITE,BRAND_NAME,SITE_NO,CATE_NAMES,BRAND,SELL
657811,490,0,0,30577519002,9933893,2018-07-13 13:42:50,2018-06-08,FILTER ALGO,2018-06-08,30577519,[메이블린 뉴욕]_핏미 파운데이션+컨실러 2종세트,http://www.gsshop.com/prd/prd.gs?prdid=30577519,23200,메이블린 뉴욕,1,메이크업>베이스,LOREAL,1.1368e+07
657956,481,0,0,30577519007,9933893,2018-07-13 13:42:50,2018-06-08,FILTER ALGO,2018-06-08,30577519,[메이블린 뉴욕]_핏미 파운데이션+컨실러 2종세트,http://www.gsshop.com/prd/prd.gs?prdid=30577519,23200,메이블린 뉴욕,1,메이크업>베이스,LOREAL,1.11592e+07
658101,499,0,0,30577519012,9933893,2018-07-13 13:42:50,2018-06-08,FILTER ALGO,2018-06-08,30577519,[메이블린 뉴욕]_핏미 파운데이션+컨실러 2종세트,http://www.gsshop.com/prd/prd.gs?prdid=30577519,23200,메이블린 뉴욕,1,메이크업>베이스,LOREAL,1.15768e+07
658246,498,0,0,30577519017,9933893,2018-07-13 13:42:50,2018-06-08,FILTER ALGO,2018-06-08,30577519,[메이블린 뉴욕]_핏미 파운데이션+컨실러 2종세트,http://www.gsshop.com/prd/prd.gs?prdid=30577519,23200,메이블린 뉴욕,1,메이크업>베이스,LOREAL,1.15536e+07
658391,500,0,0,30577519022,9933893,2018-07-13 13:42:50,2018-06-08,FILTER ALGO,2018-06-08,30577519,[메이블린 뉴욕]_핏미 파운데이션+컨실러 2종세트,http://www.gsshop.com/prd/prd.gs?prdid=30577519,23200,메이블린 뉴욕,1,메이크업>베이스,LOREAL,1.16e+07
658536,500,0,0,30577519027,9933893,2018-07-13 13:42:50,2018-06-08,FILTER ALGO,2018-06-08,30577519,[메이블린 뉴욕]_핏미 파운데이션+컨실러 2종세트,http://www.gsshop.com/prd/prd.gs?prdid=30577519,23200,메이블린 뉴욕,1,메이크업>베이스,LOREAL,1.16e+07


In [ ]:
old[(old.ITEM_ID==8764787)&(old.STOCK_ID=='-1251111349027131874')]

In [ ]:
OLI[(OLI.ITEM_ID==11961831)&(OLI.STOCK_ID=='7405510148804182974')]

In [ ]:
temp = OLI.groupby("ITEM_ID").STOCK_AMOUNT.nunique()
st = temp[temp==1].reset_index()
temp2 = old[old.ITEM_ID.isin(st.ITEM_ID)]
temp3 = temp2[temp2.STOCK_AMOUNT.isin([99,999,9999,99999,999999,9999999])]
temp3.STOCK_AMOUNT.unique()

In [ ]:
temp = GSS.groupby("ITEM_ID").STOCK_AMOUNT.nunique()
st = temp[temp==1].reset_index()
temp2 = GSS[GSS.ITEM_ID.isin(st.ITEM_ID)]
temp3 = temp2[temp2.STOCK_AMOUNT.isin([99,999,9999,99999,999999,9999999])]
temp3.STOCK_AMOUNT.unique()

In [ ]:
temp3.groupby()

In [ ]:
temp = HMA.groupby("ITEM_ID").STOCK_AMOUNT.nunique()
st = temp[temp==1].reset_index()
temp2 = HMA[HMA.ITEM_ID.isin(st.ITEM_ID)]
temp3 = temp2[temp2.STOCK_AMOUNT.isin([99,999,9999,99999,999999,9999999])]
temp3.STOCK_AMOUNT.unique()

## URL 뽑기

In [ ]:
AP = "12819911 2313319 7563091 3856973 12081822 12819947 12819943 12819948 12081825 12803503".split(" ")
LG = "3059399 4044564 3550578 3857804 2030249 4044557 3855427 3858137 2032728 2729103".split(" ")
LOREAL = "12850037 12850038 12808816 1838281 8845990 10365990".split(" ")
LVMH = "12803502 3854124 12795432 1807377 3236579 12803524 1835433 11548531 1852738 12752682".split(" ")

In [ ]:
AP = [int(i) for i in AP]
LG = [int(i) for i in LG]
LOREAL = [int(i) for i in LOREAL]
LVMH = [int(i) for i in LVMH]

In [ ]:
l1 = HMA[HMA.ITEM_ID.isin(AP)][['COLLECT_URL', 'BRAND']].drop_duplicates()
l2 = HMA[HMA.ITEM_ID.isin(LG)][['COLLECT_URL', 'BRAND']].drop_duplicates()
l3 = HMA[HMA.ITEM_ID.isin(LOREAL)][['COLLECT_URL', 'BRAND']].drop_duplicates()
l4 = HMA[HMA.ITEM_ID.isin(LVMH)][['COLLECT_URL', 'BRAND']].drop_duplicates()

In [ ]:
pd.concat([l1,l2,l3,l4]).to_csv("HMA.csv")

In [ ]:
AP = "8750156 12721985 12721996 12794053 12722000 12756680 12597033 12609406 1445434 12733865".split(" ")
LG = "12625490 12625344 12180291 12625467 12625354 12625461 12392147 12625393 12625392 12755019".split(" ")
LOREAL = "5312607 5312588 5319995 5312598 5312606 4338237 5319964 4337644 5320003 1760318".split(" ")
LVMH = "12722022 12625534 12625535 12834127 12834129 12625253 6672884 12834128 12834130".split(" ")

In [ ]:
AP = [int(i) for i in AP]
LG = [int(i) for i in LG]
LOREAL = [int(i) for i in LOREAL]
LVMH = [int(i) for i in LVMH]

In [ ]:
l1 = GSS[GSS.ITEM_ID.isin(AP)][['COLLECT_URL', 'BRAND']].drop_duplicates()
l2 = GSS[GSS.ITEM_ID.isin(LG)][['COLLECT_URL', 'BRAND']].drop_duplicates()
l3 = GSS[GSS.ITEM_ID.isin(LOREAL)][['COLLECT_URL', 'BRAND']].drop_duplicates()
l4 = GSS[GSS.ITEM_ID.isin(LVMH)][['COLLECT_URL', 'BRAND']].drop_duplicates()

In [ ]:
pd.concat([l1,l2,l3,l4]).to_csv("GSS.csv")

In [ ]:
l2

In [ ]:
GSS[GSS.ITEM_ID==12819911]